# Plink 1.07
* https://mopipe.tistory.com/24?category=820817 참고

* tutorial 
01. getting started
* plink --file hapmap1 # 해당하는 파일에 대한 대략적인 summary를 나타냄
 -> hapmap1.ped  hapmap1.map 이 필요함 
 -> plink --vcf @file --out @filename --recode (ped,map 생성)
 -> plink --vcf @file --out @filename --make-bed ( bim, bed, fam 형성)
* ped file = genotype information
  map = name, position, marker등의 메타 데이터를 가짐
* log 확인을 통하여 각각의 데이터의 구성을 알 수 있음

In [ ]:
## making a binary PED file (용량을 위하여 이진 파일로 시행)
plink --file hapmap1 --make-bed --out hapmap1 #bim, bed, fam파일을 형성


## binary PED file 이용하여 summary시행 (위의 --file과 비슷)
plink --bfile hapmap1 

## missin data에 대한 summary를 얻기 (없는 것을 의미)
plink --bfile hapmap1 --missing --out miss_stat
# miss_stat.imiss = individual missingness information 개인별 분석 결과
# miss_stat.lmiss = locus missingness information snp별 분석 결과

## allele frequency계산
plink --bfile hapmap1 --freq --out freq_stat
# freq_stat.frq의 allele frequency계산
# CLST의 컬럼의 경우 인종별 존재하는 pop.phe file중 일본,중국중 어디에 포함이 되는지 나타낸다.
plink --bfile hapmap1 --snp rs1891905 --freq --within pop.phe --out snp1 frq stat
# 특정 snp가 인종별로 어떻게 나타나는지 확인

In [ ]:
## basic association analysis 
plink --bfile hapmap1 --assoc --out as1
# snp, A1은 적은 allele, F_A : case에서 비율, F_U : control에서 비율

## p-value를 기준으로 가장 최상의 값들만 추출을 시행(리눅스 언어)
sort --key=7 -nr as1.assoc | head 

## hypothesis 검정을 시행 (FDR)
plink --bfile hapmap1 --assoc --adjust --out as2 
# allele frequency가 비슷해도 위의 데이터에서 유의하게 나오는 경우가 존재함 
# as2.assoc.adjusted에 저장되어 짐
# bonferroni, FDR, GC(inflation factor)등 모두 계산을 시행한 값이 나옴~!!~
# log data에 관련되어진 inflation factor와 chi squared value 존재
plink --bfile hapmap1 --pheno pop.phe --assoc --adjust --out as3
# 이미 인구집단과 관련되어진 자료가 존재 할 시 보정 가능

In [ ]:
https://doc.goldenhelix.com/SVS/latest/svsmanual/genotype_association_tests.html 
각각의 모델에 대한 설명이 존재함    
    
## Genotypic and other association models
plink --bfile hapmap1 --model --snp rs2222162 --out mod1
# 관련되어진 snp을 찾았으면 이제 경향성을 파악하기 위하여 cochrane -armitage test를 시행한다. 
# 2x3데이터 테이블을 가지고 분석을 진행 
# dominant, recessive 의 경우 모두 분석이 가능함

### 결과는 각각의 test에 따른 결과를 나타낸다.
#genotypic test/ basic allelic/ cochrean / dominant/ recessive 
plink --bfile hapmap1 --model --cell 0 --snp rs2222162 --out mod2
# 위의 --cell의 옵션이 존재하지 않을 시 genotypic 분석이 되지 않음 (5개 이상으로만 시행이 되어서 )

In [ ]:
## stratification analysis
# sample들 간의 연관성이 존재하기 때문에 이를 파악하여 그룹화를 시행 가능함
plink --bfile hapmap1 --cluster --mc 2 --ppc 0.05 --out str1
# --mc 2명이상 군집화를 하지 않으며 0.05이하의 유의성을 가지는 개인으로 군집활르ㅜ 시행

In [ ]:
## association analysis, accounting for clusters
# cochran-mantel-Haenszel association statistic을 사용하여 진행
plink --bfile hapmap1 --mh --within str1.cluster2 --adjust --out aac1

In [ ]:
## pc의 필터링을 적게해서 시행 (version.cluster1생성)
# specify the threshold of 0.01로 잡음
# --cc 옵션도 추가되어짐
plink --bfile hapmap1 --cluster --cc --ppc 0.01 --out version2
plink --bfile hapmap1 --mh --within version2.cluster2 --adjust --out aac2  # 유의한 결과가 발생함 (acc2.cmh.adjusted)

## p.35번 부터 시작해